In [1]:
import os
import pytorch_lightning as pl

import glob
import pandas as pd
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
from sklearn.model_selection import KFold, StratifiedKFold

from data_loader import CustomDataset
from sklearn import preprocessing

In [2]:
all_img_list = glob.glob('./aug_data/train/*/*')
df = pd.DataFrame(columns=['img_path', 'label'])
df['img_path'] = all_img_list
df['label'] = df['img_path'].apply(lambda x : str(x).split('/')[-2])

kf = StratifiedKFold(n_splits=5,
            shuffle=True,
            random_state=41)

all_splits = [k for k in kf.split(df, df['label'])]
train_idx, val_idx = all_splits[0]
train_idx, val_idx = train_idx.tolist(), val_idx.tolist()

In [3]:
print(f'all splits length: [{len(all_splits)}]')
print(f'all splits[0][0] shape: [{all_splits[0][0].shape}]') # 80%
print(f'all splits[0][1] shape: [{all_splits[0][1].shape}]') # 20%

all splits length: [5]
all splits[0][0] shape: [(2896,)]
all splits[0][1] shape: [(724,)]


In [4]:
print(f'df shape from train_idx : [{df.iloc[train_idx].shape}]')
print(f'df shape from val_idx : [{df.iloc[val_idx].shape}]')

train = df.iloc[train_idx]
val = df.iloc[val_idx]

df shape from train_idx : [(2896, 2)]
df shape from val_idx : [(724, 2)]


ValueError: cannot set a row with mismatched columns

In [6]:
train[train.label == 16]

,img_path,label
1663,./aug_data/train/틈새과다/3.png,16
1664,./aug_data/train/틈새과다/2.png,16
1665,./aug_data/train/틈새과다/0.png,16
1667,./aug_data/train/틈새과다/1.png,16


In [7]:
train.label.value_counts()

18    1100
10     469
1      253
3      175
15     153
2      150
11     119
7      108
6       86
9       48
5       46
17      44
12      43
14      28
13      22
8       22
4       17
0        9
16       4
Name: label, dtype: int64

In [8]:
val.label.value_counts()

18    275
10    118
1      64
3      44
15     39
2      37
11     30
7      26
6      21
9      12
5      11
12     11
17     11
8       6
14      6
13      5
4       4
0       3
16      1
Name: label, dtype: int64

In [ ]:
train_dataset = CustomDataset(train['img_path'].values, train['label'].values, train_transform_4_origin)